In [1]:
from helpers import *

In [2]:
df_datas = create_data()

NameError: name 'create_data' is not defined

In [3]:
dfs = []
for r, d, f in os.walk(os.getcwd()):
    for file in f:
        if 'withheldtweets.json' in file:
            dfs.append(pd.read_json("./censored_tweets/%s" % file, lines=True))

df_cen = pd.concat(dfs)
df_cen = df_cen.dropna(subset=['withheld_in_countries'])

In [5]:
df_cen.head(100)

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,filter_level,lang,timestamp_ms,linked,display_text_range,withheld_in_countries,extended_entities,possibly_sensitive,retweeted_status,withheld_copyright
1,2021-02-26 10:48:35+00:00,1365252415444946945,1365252415444946944,#Balakot \nPak Army is our pride ❤️👍 https://t...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,en,2021-02-26 10:48:35.662,no,"[0, 35]",[IN],"{'media': [{'id': 1365252409015033857, 'id_str...",0.0,NaN,NaN
2,2021-02-26 10:45:51+00:00,1365251727574900738,1365251727574900736,RT @ZaidZamanHamid: لوگوں کی اکثریت ہدایت نہیں...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,ur,2021-02-26 10:45:51.661,no,NaN,[IN],NaN,NaN,{'created_at': 'Thu Feb 25 18:59:12 +0000 2021...,NaN
3,2021-02-26 10:45:51+00:00,1365251727574900738,1365251727574900736,RT @ZaidZamanHamid: لوگوں کی اکثریت ہدایت نہیں...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,ur,2021-02-26 10:45:51.661,retweeted,NaN,[IN],NaN,NaN,{'created_at': 'Thu Feb 25 18:59:12 +0000 2021...,NaN
4,2021-02-26 10:38:57+00:00,1365249991137251328,1365249991137251328,RT @SaniaNishtar: سید ابراہیم کا تعلق قبائلی ض...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,ur,2021-02-26 10:38:57.662,no,NaN,[IN],NaN,NaN,{'created_at': 'Fri Feb 26 05:23:00 +0000 2021...,NaN
5,2021-02-26 10:10:45+00:00,1365242894362279938,1365242894362279936,RT @mosa_abumarzook: في مثل فجر هذا اليوم قبل ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,...,low,ar,2021-02-26 10:10:45.659,no,NaN,[IL],NaN,NaN,{'created_at': 'Thu Feb 25 19:04:40 +0000 2021...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,2021-02-27 03:25:19+00:00,1365503251592843266,1365503251592843264,RT @iEsraBilgic: پی ٹی آئی کے ٹائیگرز اپنے آپک...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,ur,2021-02-27 03:25:19.660,no,NaN,[IN],NaN,NaN,{'created_at': 'Fri Feb 26 16:48:27 +0000 2021...,NaN
98,2021-02-27 03:25:28+00:00,1365503289329033218,1365503289329033216,RT @MAslam_k: What A Day Was \n#PakRetaliation...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,en,2021-02-27 03:25:28.657,no,NaN,[IN],"{'media': [{'id': 1365497157478973445, 'id_str...",0.0,{'created_at': 'Sat Feb 27 03:01:07 +0000 2021...,NaN
99,2021-02-27 03:59:36+00:00,1365511879288893440,1365511879288893440,RT @MasudAKhan6: With my son. 😚 Good fathers m...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,en,2021-02-27 03:59:36.663,no,NaN,[IN],"{'media': [{'id': 1365306695866540034, 'id_str...",0.0,{'created_at': 'Fri Feb 26 14:24:19 +0000 2021...,NaN
100,2021-02-27 03:59:36+00:00,1365511879288893440,1365511879288893440,RT @MasudAKhan6: With my son. 😚 Good fathers m...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,en,2021-02-27 03:59:36.663,retweeted,NaN,[IN],"{'media': [{'id': 1365306695866540034, 'id_str...",0.0,{'created_at': 'Fri Feb 26 14:24:19 +0000 2021...,NaN


In [16]:
pd.options.display.max_colwidth = 500
pd.options.display.width = 300
df_english = df_cen[df_cen['lang'] == 'en']
df_english['text']

1                                                                                        #Balakot \nPak Army is our pride ❤️👍 https://t.co/JirGPSMMGO
11       RT @Saj_PakPassion: Ben Dunk "If you haven't seen Lahore, you haven't even been born. It was my first time in Lahore last year. The home fa…
15     RT @ItzLovelyKhan: #ہماراایمان_دفاع_پاکستان\n\nIndia has also bestowed Abhinandan Varthaman with its 3rd-highest military award. India awarde…
28       RT @HT_KENYA: After the great Companion Saad bin Muadh embraced Islam, he went to his people - said to them: Its forbidden for me to listen…
29       RT @HT_KENYA: After the great Companion Saad bin Muadh embraced Islam, he went to his people - said to them: Its forbidden for me to listen…
                                                                            ...                                                                      
609    RT @goals557: Ba rca x P S G🔥\n\nKindly retweet and follow @mrfc100 to enjoy all games \n\nRe